In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
pso_df = pd.read_excel("C:\\Users\\Nefeli\\Desktop\\opti2_stat_scripts\\pso_results\\pso_full.xlsx")
ga_df = pd.read_excel("C:\\Users\\Nefeli\\Desktop\\opti2_stat_scripts\\ga_results\\ga_full.xlsx")

In [3]:
def excelListCleanup(inputDfRow):
    toClean = inputDfRow
    toRemove=['[',']','\n'] 
    for i in toRemove:
        toClean = toClean.replace(i,'')
    toClean = toClean.split(' ')
    toClean = [i for i in toClean if i!='']
    toClean = [float(i)for i in toClean]
    #print(toClean)
    return toClean
#params_df.iat[65, params_df.columns.get_loc('costVal_list')]
#print(excelListCleanup(params_df.iat[65, params_df.columns.get_loc('costVal_list')]))

In [4]:
pso_df.costVal_list = pso_df.costVal_list.apply(lambda x : excelListCleanup(x))
pso_df.feasibility_list = pso_df.feasibility_list.apply(lambda x : excelListCleanup(x))
pso_df.lastHit_list = pso_df.lastHit_list.apply(lambda x : excelListCleanup(x))

ga_df.costVal_list = ga_df.costVal_list.apply(lambda x : excelListCleanup(x))
ga_df.feasibility_list = ga_df.feasibility_list.apply(lambda x : excelListCleanup(x))
ga_df.lastHit_list = ga_df.lastHit_list.apply(lambda x : excelListCleanup(x))

In [5]:
best_pso = pso_df[(pso_df['nb_r'] == 5) & (pso_df['N'] == 200)]

In [6]:
best_ga = ga_df[(ga_df['select_op'] == 25) & (ga_df['N'] == 200)]

In [7]:
def kolmogorovSmirnovTailed(F,G,alternative, a):
    result = stats.ks_2samp(F,G,alternative)
    #compare the p value with the significance level a
    if result[1]<a:
        return 1  #null hypothesis REJECTED -> cdf(F)>cdf(G)
    else:
        return 0  #null hypothesis RETAINED -> cdf(F)<=cdf(G)

In [12]:
print("KS Test PSO vs GA score (cost val): ",kolmogorovSmirnovTailed(best_pso.costVal_list.values[0],best_ga.costVal_list.values[0],'greater',0.05)) 
print("KS Test GA vs PSO score (cost val): ",kolmogorovSmirnovTailed(best_ga.costVal_list.values[0],best_pso.costVal_list.values[0],'greater',0.05))

KS Test PSO vs GA score (cost val):  0
KS Test GA vs PSO score (cost val):  1


In [13]:
print("KS Test PSO vs GA score (last hit): ",kolmogorovSmirnovTailed(best_pso.lastHit_list.values[0],best_ga.lastHit_list.values[0],'greater',0.05)) 
print("KS Test GA vs PSO score (last hit): ",kolmogorovSmirnovTailed(best_ga.lastHit_list.values[0],best_pso.lastHit_list.values[0],'greater',0.05))

KS Test PSO vs GA score (last hit):  0
KS Test GA vs PSO score (last hit):  0


### Wilcoxon Rank Sum Test aka Mann Whitney U test 

In [15]:
from scipy.stats import ranksums
# Perform Wilcoxon Rank Sum Test
statistic, p_value = ranksums(best_pso.costVal_list.values[0],best_ga.costVal_list.values[0])

# Print the test statistic and p-value
print("Wilcoxon Rank Sum Test (Cost Values)")
print("P-value:", p_value)
print("H0 Rejection:", p_value<0.05)

Wilcoxon Rank Sum Test Statistic: 4.045494225606034
P-value: 5.221285277796408e-05
H0 Rejection: True


In [18]:
# Perform Wilcoxon Rank Sum Test
statistic, p_value = ranksums(best_pso.lastHit_list.values[0],best_ga.lastHit_list.values[0])

# Print the test statistic and p-value
print("Wilcoxon Rank Sum Test (Last Hit)")
print("P-value:", p_value)
print("H0 Rejection:", p_value<0.05)

Wilcoxon Rank Sum Test (Last Hit)
P-value: 0.3567187999388043
H0 Rejection: False
